## DistilBERT Fine-Tuning 

In [16]:
import torch
from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertTokenizer, BertModel, DistilBertForSequenceClassification, DistilBertTokenizer

import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import helpers as helpers
import torch.nn as nn

In [11]:
# DistilBERT: max_len=128–256, batch_size=16, lr≈2e-5, epochs=2–3, early stopping.

DEPRESSION_PATH = "data/depression_dataset_reddit_cleaned.csv"
TDT_SPLIT = "80/10/10"
BERT_MODEL = 'distilbert-base-uncased'

MAX_LENGTH = 256
BATCH_SIZE = 16
NUM_EPOCHS = 3
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 0.01

In [3]:
data, labels = helpers.get_data()
train_text, dev_text, test_text = helpers.split(data=data, dist=TDT_SPLIT)
train_dep, dev_dep, test_dep = helpers.split(data=labels, dist=TDT_SPLIT)

# convert data, labels into a pandas DataFrame
df = pd.DataFrame({'clean_text': data, 'is_depression': labels})

Completing 80/10/10 split
Completing 80/10/10 split


In [4]:
# initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained(BERT_MODEL)
bert = DistilBertForSequenceClassification.from_pretrained(BERT_MODEL, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# stats from tokenizers
print("Vocab length is : ",len(tokenizer.vocab))
print(tokenizer.decode(2897))

vocab = list(tokenizer.vocab.keys())
print(f"first 10: {vocab[:10]}")
print(f"1280-1290: {vocab[1280:1290]}")
print(f"last 10: {vocab[-10:]}")

print(f"token 2897: {tokenizer.decode(2897)}")
print(f"token 102: {tokenizer.decode(102)}")
print(f"token 103: {tokenizer.decode(103)}")
if 'alex' in tokenizer.vocab:
  print(f"alex id: {tokenizer.vocab['alex']}")
else:
  print("alex not found")

Vocab length is :  30522
network
first 10: ['[PAD]', '[unused0]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]']
1280-1290: ['ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ']
last 10: ['##！', '##（', '##）', '##，', '##－', '##．', '##／', '##：', '##？', '##～']
token 2897: network
token 102: [SEP]
token 103: [MASK]
alex id: 4074


In [6]:
# tokenize train, dev, and test
train_encodings = tokenizer(train_text, truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors="pt")
dev_encodings = tokenizer(dev_text, truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors="pt")
test_encodings = tokenizer(test_text, truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors="pt")

In [7]:
# Convert labels to tensors
train_labels = torch.tensor(train_dep)
dev_labels = torch.tensor(dev_dep)
test_labels = torch.tensor(test_dep)

# make TensorDatasets
train_dataset = TensorDataset(
    train_encodings['input_ids'],
    train_encodings['attention_mask'],
    train_labels
)

dev_dataset = TensorDataset(
    dev_encodings['input_ids'],
    dev_encodings['attention_mask'],
    dev_labels
)

test_dataset = TensorDataset(
    test_encodings['input_ids'],
    test_encodings['attention_mask'],
    test_labels
)

# Create DataLoaders
dl_train = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dl_dev = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False)
dl_test = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [17]:
optimizer = AdamW(bert.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
# begin training

for e in range(NUM_EPOCHS):
    print()